# Дембицкий М.Е. ДЗ № 3

## Оглавление
[Задание 1](#task1)  
[Задание 2](#task2)  
[Задание 3](#task3)  
[Задание 4](#task4)  


### Домашнее задание

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [47]:
# Подгружаем библиотеки

import numpy as np
import pandas as pd

# ml
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix

import matplotlib.pyplot as plt

%matplotlib inline

In [19]:
# Подгружаем датасет
df = pd.read_csv('train_case2.csv', ';')
df.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [20]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1), 
                                                    df['cardio'], random_state=0)

In [21]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [22]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

<a name="task1"></a>
### Задание 1 
 
Обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта

Я буду использовать LGBMClassifier, LogisticRegression и RandomForestClassifier


<a name="task2"></a>
### Задание 2 
 При обучении моделей обязательно использовать кроссвалидацию

In [43]:
log_classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

random_forest_classifier = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

lgbm_classifier = Pipeline([
    ('features',feats),
    ('classifier', LGBMClassifier(random_state = 42)),
])

classifier_list = [lgbm_classifier, log_classifier, random_forest_classifier]

#запустим кросс-валидацию
for model in classifier_list:   
    cv_scores = cross_val_score(model, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    print(f'CV score of {model[1]} is {cv_score} +- {cv_score_std}')

#обучим пайплайны на всех тренировочных датасетах
models_y_scores = {}
for model in classifier_list:
    model.fit(X_train, y_train)
    y_score = model.predict_proba(X_test)[:, 1]
    models_y_scores[model[1]] = y_score

CV score of LGBMClassifier(random_state=42) is 0.8024230111465757 +- 0.006885787188310477
CV score of LogisticRegression(random_state=42) is 0.7867401104915408 +- 0.00852135511666111
CV score of RandomForestClassifier(random_state=42) is 0.7734501681056019 +- 0.007171140345435727


Пока что LGBMClassifier выдает лучший скор

In [59]:
# Посчитаем precision/recall/f_score, roc auc для наших моделей
data = []
b=1
for k, v in  models_y_scores.items():
    precision, recall, thresholds = precision_recall_curve(y_test.values, v)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
#     roc_auc = roc_auc_score(y_true=y_test, v)
    print(f'{k}:', 'Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, Roc Auc=%.3f' % (thresholds[ix], 
                                                                            fscore[ix],
                                                                            precision[ix],
                                                                            recall[ix],
                                                                            roc_auc_score(y_test, v)))

    metrics_data = [precision[ix], recall[ix], fscore[ix], roc_auc_score(y_test, v)]
    data.append(metrics_data)

LGBMClassifier(random_state=42): Best Threshold=0.344797, F-Score=0.740, Precision=0.661, Recall=0.840, Roc Auc=0.802
LogisticRegression(random_state=42): Best Threshold=0.386937, F-Score=0.730, Precision=0.647, Recall=0.838, Roc Auc=0.784
RandomForestClassifier(random_state=42): Best Threshold=0.350000, F-Score=0.719, Precision=0.643, Recall=0.816, Roc Auc=0.771


<a name="task3"></a>
### Задание 3
вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)



In [61]:
cols = ['precision', 'recall', 'fscore', 'roc auc score']
rows = ['lgbm_classifier', 'log_classifier', 'random_forest_classifier']

concl = pd.DataFrame(data, rows, cols)
concl

,precision,recall,fscore,roc auc score
lgbm_classifier,0.660803,0.839631,0.739561,0.801689
log_classifier,0.647431,0.837558,0.730323,0.784035
random_forest_classifier,0.642669,0.815553,0.718863,0.771037


<a name="task4"></a>
### Задание 4

Сделать вывод о том, какая модель справилась с задачей лучше других

**Вывод**  
Из 3 рассмотренных моделей классификации, лучше всего себя показала LGBMClassifier, на втором месте Модель Логистической регрессии и хуже всего себя показал Random Forest, он же обрабатывал данные дольше всего.